In [1]:
import json
%load_ext rich
import datasets as ds
import numpy as np
import pandas as pd

/home/grisha/virtual_env/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("role-mapping.json", 'r') as f:
    role_mapping = json.loads(f.read())

In [3]:
import spacy
model = spacy.load("ru_core_news_lg")

In [4]:
def spacy_lemmatize(word):
    return next(iter(model(word))).lemma_

In [48]:
data = ds.load_dataset("Rexhaif/framebank_srl")['train']

In [6]:
def filter_tagged(example):
    return any(example['srl'])

In [49]:
data = data.filter(filter_tagged)

In [8]:
def get_predicate(example):
    predicate = example['lemmas'][example['rank'].index('Предикат')]
    return predicate

def get_sentence(example):
    text = " ".join(example['tokens'])
    predicate = example['tokens'][example['rank'].index('Предикат')]
    for sent in model(text).sents:
        if predicate in str(sent):
            return str(sent)
    raise Exception("Predicate not found!")

def remapper(example):
    return {
        'text_fixed':  get_sentence(example),
        'predicate': get_predicate(example),
    }

In [50]:
data = data.map(remapper, num_proc=16)

In [41]:
def register_predicates(ds):
    predicates_count = {}
    def handle_predicate(example):
        predicate = get_predicate(example)
        if predicate not in predicates_count:
            predicates_count[predicate] = 1
        else:
            predicates_count[predicate] += 1
        return example

    handle_predicate(ds[0])
    ds.map(handle_predicate)
    print(predicates_count)
    return predicates_count

In [42]:
predicates_count = register_predicates(data)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████| 32631/32631 [00:02<00:00, 12709.61 examples/s]

{'зависеть': 175, 'бесить': 144, 'добиваться': 130, 'брызгать': 124, 'беспокоить': 118, 'мочь': 115, 'соблюдать': 111, 'гладить': 110, 'угодить': 108, 'догонять': 106, 'любоваться': 106, 'метаться': 106, 'чистить': 106, 'беседовать': 104, 'виднеться': 103, 'объяснять': 103, 'переспросить': 102, 'радовать': 102, 'стыдиться': 101, 'велеть': 100, 'выбежать': 99, 'выглядеть': 99, 'заявлять': 99, 'носиться': 99, 'рассчитывать': 99, 'потребовать': 99, 'возиться': 98, 'вспомнить': 98, 'вынуть': 98, 'поморщиться': 98, 'оценить': 98, 'пожать': 98, 'появиться': 98, 'влюбиться': 97, 'жечь': 97, 'объясняться': 97, 'оставаться': 97, 'пытаться': 97, 'становиться': 97, 'теряться': 97, 'делать': 96, 'исчезать': 96, 'менять': 96, 'свести': 96, 'ухаживать': 96, 'победа': 96, 'завести': 95, 'звать': 95, 'мечтать': 95, 'отступить': 95, 'появляться': 95, 'превратиться': 95, 'претендовать': 95, 'ссориться': 95, 'увести': 95, 'являться': 95, 'продлиться': 95, 'глядеть': 94, 'прогреметь': 94, 'демонстрировать

In [45]:
good_predicates = [pred for (pred, cnt) in predicates_count.items() if cnt >= 10 and pred in role_mapping.keys()]

In [13]:
def filter_fn(example):
    predicate = get_predicate(example)
    return predicate in good_predicates and all([role in role_mapping[predicate] for role in example['srl'] if role is not None])

In [51]:
data = data.filter(filter_fn)

In [15]:
import datasets as ds
from tqdm.auto import tqdm, trange

In [16]:
def extract_examples(ds, n_examples=5):
    examples = {}
    filled_roles = {}

    def pick_as_example(example):
        predicate = get_predicate(example)
        if predicate not in examples:
            examples[predicate] = []
            filled_roles[predicate] = {}

        new_roles = False
        for role in example['srl']:
            if not role in filled_roles[predicate]:
                filled_roles[predicate][role] = True
                new_roles = True

        exclude = False
        if len(examples[predicate]) < n_examples or new_roles:
            examples[predicate].append(example)
            exclude = True
        return {'exclude': exclude}

    pick_as_example(ds[0])
    new_ds = ds.map(pick_as_example)
    new_ds = new_ds.filter(lambda example: not example['exclude'])
    return new_ds, examples

In [72]:
data = data.shuffle(seed=42)
data, examples_for_predicates = extract_examples(data)

Filter: 100%|████████████████████████████████████████████████████████████████████████████████| 25090/25090 [00:01<00:00, 21297.49 examples/s]


In [86]:
def make_prompt_for_example_vllm(example):
    example_predicate = get_predicate(example)
    rule_set = role_mapping[example_predicate]
    rule_set = json.dumps(rule_set, ensure_ascii=False, indent=4)
    example_set = examples_for_predicates[example_predicate]

    prompt = [
        {
            'role': 'system',
            'content': f'''
You are a native Russian linguist specializing in semantic role labelling. \
You must find all the arguments of a given verb in the sentence \
and assign each argument a role from the given list.

Instructions:
- Do not mark semantic roles for implied, implicit or otherwise not presented arguments
- Reason out loud before answering
- Examine all the possible arguments carefully before making a conclusion
- If the argument is a phrase, find the main indicative word of that phrase. \
The main indicative word is the one that SEMANTICALLY defines the phrase, \
e.g. if the phrase is "что ещё есть время", the main word is "есть" and not "что".
- Overall, the argument should be represented by ONE SINGLE WORD in EXACTLY \
the same form as it had in the sentence.
- If several homogeneous words fit the role (e.g. "жизнь и смерть" or \
"трубочист Иван Петрович"), always choose the first one ("жизнь", "трубочист").
- After you're done reasoning, you should preface your answer with a \
single line saying "Here you are!" and then give your answer exactly in the form \
"argument#role\\n" for every argument that you have found.
- If there are no semantic roles for any argument that you can extract, don't \
write anything after the preface phrase.
''',
#                         'content': f'''
# You are a native Russian linguist specializing in semantic role labelling. \
# You must find all the arguments of a given verb in the sentence \
# and assign each argument a role from the given list.

# Instructions:
# - Do not mark semantic roles for implied, implicit or otherwise not presented arguments
# - If the argument is a phrase, find the main indicative word of that phrase. \
# - The argument should be represented by ONE SINGLE WORD in EXACTLY the same form as it had in the sentence.
# - After you're done reasoning, you should preface your answer with a \
# single line saying "Here you are!" and then give your answer exactly in the form \
# "argument#role\\n" for every argument that you have found.
# - If there are no semantic roles for any argument that you can extract, don't \
# write anything after the preface phrase.
# ''',
        }
    ]
    examples = ""
    for ex in example_set:
        examples += f"Example Text:\n{ex['text_fixed']}\n"

        semantic_roles = ""
        for (word, role) in zip(ex['tokens'], ex['srl']):
            if role is not None:
                semantic_roles += f'{word}#{role}\n'
        examples += f"Example Semantic Roles:\n{semantic_roles}\n\n"

    inputs = f'''
Given a series of few-shot examples, please find all the arguments of the predicate "{example_predicate}" \
and label them with semantic roles from the following list: ```json{rule_set}```.
Here are the few-shot examples:

{examples}

Here is the target sentence:
{example['text_fixed']}

Please list all the arguments that you find in the format described above.

'''.strip()


    prompt.append({
        'role': 'user',
        'content': inputs
    })
    return prompt

In [70]:
examples_for_predicates.keys()

dict_keys(['отрезать'])

In [ ]:
random_example = data[0]
prompt = make_prompt_for_example_vllm(random_example)
print(prompt[0]['content'], prompt[1]['content'])

In [81]:
def roles_to_str(roles):
    return '\n'.join([f'{arg}#{role}' for (arg, role) in roles.items()])

In [135]:
from openai import OpenAI

def make_request_openai(example):
    messages = make_prompt_for_example_vllm(example)
    client = OpenAI(
        base_url = 'https://openrouter.ai/api/v1',
        api_key='',
    )

    try:
        response = client.chat.completions.create(
          model="mistralai/mistral-medium-3",
          messages=messages,
        ).choices[0].message.content
    except Exception as e:
        return {
            'llm-response': f'Error: {e}',
            'llm-roles': None,
            'true-roles': None,
            'match': False
        }
    try:
        roles = {}
        prefaced = False
        for line in response.splitlines():
            line = line.strip()
            if prefaced:
                if line == "":
                    continue
                (arg, role) = tuple(line.split(sep='#'))
                roles[arg.lower()] = role.lower()
            if line == "Here you are!":
                prefaced = True

        true_roles = {}
        true_roles_str = ""
        for (arg, role) in zip(example['tokens'], example['srl']):
            if role is not None:
                true_roles[arg.lower()] = role.lower()

        return {
            'llm-response': response,
            'llm-roles': roles_to_str(roles),
            'true-roles': roles_to_str(true_roles),
            'match': roles == true_roles
        }
    except Exception as e:
        raise e
        return {
            'llm-response': response,
            'llm-roles': None,
            'true-roles': None,
            'match': False
        }

In [55]:
batch_size = 20
batches = [data.select(range(i*batch_size, (i+1)*batch_size)) for i in range(len(data) // batch_size)]

In [75]:
from IPython.display import clear_output

In [87]:
correct_examples = 0
total_examples = 0
for i in (progress := tqdm(range(518, 550))):
    result = batches[i].map(make_request_openai, num_proc=20)
    result.save_to_disk(f'responses/{i}')

    correct_examples += sum(result['match'])
    total_examples += len(result)
    progress.set_description(f'correct: {correct_examples} / {total_examples}')

    clear_output(wait=True)

correct: 385 / 640: 100%|████████████████████████████████████████████████████████████████████████████████████| 32/32 [09:07<00:00, 17.12s/it]


In [ ]:
# 0-500 Gemini
# 500-... Mistral

In [164]:
print(make_prompt_for_example_vllm(data[12])[1]['content'])

Given a series of few-shot examples, please find all the arguments of the predicate брызгать and label them with semantic roles from the following list: ```json[
    "субъект перемещения",
    "пациенс перемещения",
    "каузатор",
    "агенс",
    "конечная точка",
    "эффектор",
    "средство"
]```.
Here are the few-shot examples:

Example Text:
С громким смехом умываются и брызгаются ранние гости на лугу .
Example Semantic Roles:
гости#агенс


Example Text:
-- Дэр кричит , слюной брызгает .
Example Semantic Roles:
Дэр#каузатор
слюной#пациенс перемещения


Example Text:
Краткими залпами брызгал дождь .
Example Semantic Roles:
дождь#эффектор


Example Text:
Катер долго не может пристать , пятится , фырчит , брызгается винтом .
Example Semantic Roles:
Катер#агенс


Example Text:
Провода уже оторвались от его тела , а Гусев все подпрыгивал вместе со стулом , мотал головой и страшно ржал , брызгая кровавыми слюнями .
Example Semantic Roles:
Гусев#каузатор
слюнями#пациенс перемещения





In [88]:
result = ds.load_from_disk('responses/480')
for ex in result:
    if not ex['match']:
        # print(ex)
        print(ex['text_fixed'], ex['llm-response'], 'llm roles:', ex['llm-roles'], 'true roles:', ex['true-roles'], sep='\n')
        print()
    else:
        print('ok')

ok
ok
ok
ok
ok
ok
объявить всеобщую мобилизацию российских граждан для борьбы с врагом , перевести на военное положение экономику страны и запустить мобилизационные планы предприятий , . . .
Here you are!
планы#пациенс
llm roles:
планы#пациенс
true roles:
генерал#агенс
планы#пациенс

ok
ok
ok
ok
ok
ok
ok
-- Ну кто теперь , после того как Тимофей Георгиевич продемонстрировал свое умение музицировать , посмеет назвать его необразованным солдафоном ?
Here you are!
Тимофей#агенс
умение#атрибут
llm roles:
тимофей#агенс
умение#атрибут
true roles:
тимофей#агенс
умение#ситуация в фокусе - стимул

Мы на электричку опоздали !
Here you are!
Мы#субъект перемещения
электчку#конечная точка
llm roles:
мы#субъект перемещения
электчку#конечная точка
true roles:
мы#субъект перемещения
электричку#цель

ok
ok
Кремль продемонстрировал диалог с бизнесом , не затрагивая неприятные темы .
Here you are!
Кремль#агенс
диалог#содержание мысли
llm roles:
кремль#агенс
диалог#содержание мысли
true roles:
кремль#аген

In [128]:
results = [ds.load_from_disk(f'responses/{i}') for i in range(501, 550)]
results = ds.concatenate_datasets(results)

In [129]:
print(f'total correct: {sum(results['match'])}')

total correct: 581


In [96]:
def roles_to_dict(roles_str):
    roles = {}
    for line in roles_str.split(sep='\n'):
        line = line.strip()
        if len(line) == 0:
            continue
        (arg, role) = tuple(line.split(sep='#'))
        roles[arg.lower()] = role.lower()
    return roles

In [66]:
roles_to_dict(results[0]['llm-roles'])

{'внимание': 'причина', 'часть': 'субъект психологического состояния'}

In [130]:
role_metrics = {}
for predicate in good_predicates:
    for role in role_mapping[predicate]:
        role_metrics[role] = {'tp': 0, 'fp': 0, 'fn': 0}

for example in results:
    if example['llm-roles'] is None or example['true-roles'] is None:
        continue

    llm_roles = roles_to_dict(example['llm-roles'])
    true_roles = roles_to_dict(example['true-roles'])

    try:
        for (arg, role) in llm_roles.items():
            if role in true_roles.values():
                role_metrics[role]['tp'] += 1
            else:
                role_metrics[role]['fp'] += 1
        for (arg, role) in true_roles.items():
            if role not in llm_roles.values():
                role_metrics[role]['fn'] += 1
    except Exception as e:
        print(e)

In [131]:
def count_metrics(example):
    answer = {
        'arg_tp' : 0,
        'arg_fp' : 0,
        'arg_fn' : 0,
        'role_tp' : 0,
        'role_fp' : 0,
        'role_fn' : 0
    }
    
    if example['llm-roles'] is None or example['true-roles'] is None:
        return answer

    llm_roles = roles_to_dict(example['llm-roles'])
    true_roles = roles_to_dict(example['true-roles'])

    try:
        for (arg, role) in llm_roles.items():
            if arg in true_roles.keys():
                answer['arg_tp'] += 1
            else:
                answer['arg_fp'] += 1
            if role in true_roles.values():
                answer['role_tp'] += 1
                role_metrics[role]['tp'] += 1
            else:
                answer['role_fp'] += 1
                role_metrics[role]['fp'] += 1
        for (arg, role) in true_roles.items():
            if arg not in llm_roles.keys():
                answer['arg_fn'] += 1
            if role not in llm_roles.values():
                answer['role_fn'] += 1
                role_metrics[role]['fn'] += 1
    except Exception as e:
        print(e)

    return answer

In [111]:
count_metrics(results[0])

{'arg_tp': 2, 'arg_fp': 0, 'arg_fn': 0, 'role_tp': 2, 'role_fp': 0, 'role_fn': 0}

In [132]:
results = results.map(count_metrics, num_proc=16)

Map (num_proc=16): 100%|██████████████████████████████████████████████████████████████████████████| 980/980 [00:00<00:00, 4854.22 examples/s]


In [133]:
arg_tp = sum(results['arg_tp'])
arg_fp = sum(results['arg_fp'])
arg_fn = sum(results['arg_fn'])

arg_precision = arg_tp / (arg_tp + arg_fp)
arg_recall = arg_tp / (arg_tp + arg_fn)
arg_f1 = 2 * arg_tp / (2 * arg_tp + arg_fp + arg_fn)

role_tp = sum(results['role_tp'])
role_fp = sum(results['role_fp'])
role_fn = sum(results['role_fn'])

role_precision = role_tp / (role_tp + role_fp)
role_recall = role_tp / (role_tp + role_fn)
role_f1 = 2 * role_tp / (2 * role_tp + role_fp + role_fn)

In [134]:
print(f'arg precision: {arg_precision}')
print(f'arg recall: {arg_recall}')
print(f'arg F1: {arg_f1}')
print(f'role precision: {role_precision}')
print(f'role recall: {role_recall}')
print(f'role F1: {role_f1}')

arg precision: 0.8366279069767442
arg recall: 0.8540059347181009
arg F1: 0.8452276064610866
role precision: 0.8377906976744186
role recall: 0.8407234539089848
role F1: 0.8392545136866628


In [125]:
print(role_metrics)

{'пациенс': {'tp': 1394, 'fp': 213, 'fn': 252}, 'причина': {'tp': 767, 'fp': 107, 'fn': 96}, 'субъект психологического состояния': {'tp': 927, 'fp': 74, 'fn': 123}, 'субъект поведения': {'tp': 91, 'fp': 26, 'fn': 27}, 'агенс': {'tp': 1515, 'fp': 278, 'fn': 348}, 'результат / цель': {'tp': 77, 'fp': 9, 'fn': 6}, 'содержание высказывания': {'tp': 497, 'fp': 104, 'fn': 156}, 'способ': {'tp': 71, 'fp': 44, 'fn': 30}, 'субъект перемещения': {'tp': 767, 'fp': 79, 'fn': 111}, 'пациенс перемещения': {'tp': 292, 'fp': 46, 'fn': 73}, 'каузатор': {'tp': 35, 'fp': 10, 'fn': 22}, 'конечная точка': {'tp': 438, 'fp': 75, 'fn': 66}, 'эффектор': {'tp': 117, 'fp': 48, 'fn': 64}, 'средство': {'tp': 2, 'fp': 8, 'fn': 4}, 'субъект социального отношения': {'tp': 157, 'fp': 26, 'fn': 41}, 'пациенс социального отношения': {'tp': 37, 'fp': 36, 'fn': 12}, 'субъект ментального состояния': {'tp': 253, 'fp': 43, 'fn': 59}, 'содержание действия': {'tp': 458, 'fp': 61, 'fn': 135}, 'тема': {'tp': 944, 'fp': 84, 'fn':

In [126]:
for (role, metric) in role_metrics.items():
    role_tp = metric['tp']
    role_fp = metric['fp']
    role_fn = metric['fn']

    if role_tp + role_fp == 0 or role_tp + role_fn == 0:
        print(f'freak {role}')
        continue

    role_precision = role_tp / (role_tp + role_fp)
    role_recall = role_tp / (role_tp + role_fn)
    role_f1 = 2 * role_tp / (2 * role_tp + role_fp + role_fn)
    print(f'{role}:\t{role_precision}\t{role_recall}\t{role_f1}')

пациенс:	0.8674548848786559	0.8469015795868773	0.8570550261297264
причина:	0.8775743707093822	0.8887601390498262	0.8831318364997122
субъект психологического состояния:	0.926073926073926	0.8828571428571429	0.9039492930277914
субъект поведения:	0.7777777777777778	0.7711864406779662	0.774468085106383
агенс:	0.8449525934188511	0.8132045088566827	0.8287746170678337
результат / цель:	0.8953488372093024	0.927710843373494	0.9112426035502958
содержание высказывания:	0.826955074875208	0.7611026033690659	0.7926634768740032
способ:	0.6173913043478261	0.7029702970297029	0.6574074074074074
субъект перемещения:	0.9066193853427896	0.8735763097949886	0.8897911832946636
пациенс перемещения:	0.863905325443787	0.8	0.8307254623044097
каузатор:	0.7777777777777778	0.6140350877192983	0.6862745098039216
конечная точка:	0.8538011695906432	0.8690476190476191	0.8613569321533924
эффектор:	0.7090909090909091	0.6464088397790055	0.6763005780346821
средство:	0.2	0.3333333333333333	0.25
субъект социального отношения:	0